# Apple Store App Review Organizer!

In [1]:
!pip install app_store_scraper

In [8]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import nltk
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('slow'))
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/hoangnha218/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hoangnha218/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/hoangnha218/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hoangnha218/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hoangnha218/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
import string
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [14]:
link = 'https://apps.apple.com/us/app/piccollage-grid-story-maker/id448639966'
id = str(link.split('d')[-1])
 #appname = str(link.split('/')[-2])
id


'448639966'

In [15]:
## DEF Function

def review_organizer(link):
  global pd
  #id = str(link[-10:])
  id = str(link.split('d')[-1])
  appname = str(link.split('/')[-2])
  HULA = AppStore(country='us', app_name = appname, app_id = id)
  HULA.review(how_many=30)

  df = pd.DataFrame(np.array(HULA.reviews),columns=['review'])
  df2 = df.join(pd.DataFrame(df.pop('review').tolist()))
  df2['clean_msg']= df2['review'].apply(lambda x:remove_punctuation(x))
  df2['msg_lower']= df2['clean_msg'].apply(lambda x: x.lower())
  df_2 = df2[['msg_lower']]
  df_clean = df_2.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
  HULA_list = list(df_clean['msg_lower'])
  tokenized_sents = [word_tokenize(i) for i in HULA_list]

  score = []
  for token in tokenized_sents:
    after_tagging = nltk.pos_tag(token)
    #print (token)
    #print (after_tagging)
    def penn_to_wn(tag):
        """
        Convert between the PennTreebank tags to simple Wordnet tags
        """
        if tag.startswith('J'):
            return wn.ADJ
        elif tag.startswith('N'):
            return wn.NOUN
        elif tag.startswith('R'):
            return wn.ADV
        elif tag.startswith('V'):
            return wn.VERB
        return None

        
    sentiment = 0.0
    tokens_count = 0
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    for word, tag in after_tagging:
                wn_tag = penn_to_wn(tag)
                if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                    continue
    
                lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                if not lemma:
                    continue
    
                synsets = wn.synsets(lemma, pos=wn_tag)
                if not synsets:
                    continue
    
                # Take the first sense, the most common
                synset = synsets[0]
                swn_synset = swn.senti_synset(synset.name())
                #print(swn_synset)
    
                sentiment += swn_synset.pos_score() - swn_synset.neg_score()
                tokens_count += 1
                #score.extend(list(int(sentiment)))
    score.append(sentiment)

  pos = len(list(filter(lambda x: x >= 0, score)))/len(score)
  neg = len(list(filter(lambda x: x < 0, score)))/len(score)
  #p_ratio = len(pos)/len(score)
  #n_ratio = len(neg)/len(score)
  p_percent = "{:.2%}".format(pos)
  n_percent = "{:.2%}".format(neg)
  print( "\033[2;37;42m Positive Reviews% in this app is", p_percent, '  ')
  print( "\033[2;37;41m Negative Reviews% in this app is", n_percent, '  ')
  print('\n')


  pos_index = [i for i in range(len(score)) if score[i] >=0]
  neg_index = [i for i in range(len(score)) if score[i] <0]
  pos_index_most = [i for i in range(len(score)) if score[i] == max(score)]
  neg_index_most = [i for i in range(len(score)) if score[i] == min(score)]
  # top 5
  pos_index_5 = sorted(range(len(score)), key=lambda i: score[i])[-5:] 
  neg_index_5 = sorted(range(len(score)), key=lambda i: score[i], reverse=True)[-5:]
  pos_index_5.reverse()
  neg_index_5.reverse()

  class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
  #print(color.BOLD + 'Hello World !' + color.END)

  print(color.BOLD + '\033[2;32;43m Top 5 positive reviews score:'+ color.END)
  for i in pos_index_5:
    print(score[i])

  print( color.BOLD + '\033[2;31;43m Top 5 negative reviews score:'+ color.END)
  for i in neg_index_5:
    print(score[i])


  print('\n', color.BOLD +'\033[2;32;43m Top 5 positive reviews score:'+ color.END)
  j = 1
  for i in pos_index_5:
    print('review', j, ':',df_clean.msg_lower[i])
    j+=1


  print(color.BOLD +'\n','\033[2;31;43m Top 5 negative reviews score:'+ color.END)
  j = 1
  for i in neg_index_5:
    print('review', j, ':', df_clean.msg_lower[i])
    j+=1


  df_pos = df_clean.iloc[pos_index]
  df_neg = df_clean.iloc[neg_index]

  
  # Python program to generate WordCloud
  # importing all necessary modules
  from wordcloud import WordCloud, STOPWORDS
  import matplotlib.pyplot as plt
  import pandas as pd

  # Reads 'Youtube04-Eminem.csv' file
  #df = pd.read_csv(r"Youtube04-Eminem.csv", encoding ="latin-1")
  df = df_pos

  comment_words = ''
  stopwords = set(STOPWORDS)
  newStopWords = ['app','people','find','event']
  # adding list l to set s.
  stopwords.update(newStopWords)


  # iterate through the csv file
  for val in df.msg_lower:
    
    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()
    
    # Converts each token into lowercase
    for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()
    
    comment_words += " ".join(tokens)+" "

  wordcloud = WordCloud(width = 800, height = 800,
          background_color ='white',
          stopwords = stopwords,
          min_font_size = 10).generate(comment_words)

  # plot the WordCloud image					
  print('\n')
  print(color.BOLD +'Common Words in Positive Reviews'+ color.END)
  
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  plt.title("Common Words in Positive Reviews")

  plt.show()


  df = df_neg

  comment_words = ''
  stopwords = set(STOPWORDS)
  newStopWords = ['app','people','find', 'event']
  # adding list l to set s.
  stopwords.update(newStopWords)


  # iterate through the csv file
  for val in df.msg_lower:
    
    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()
    
    # Converts each token into lowercase
    for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()
    
    comment_words += " ".join(tokens)+" "

  wordcloud = WordCloud(width = 800, height = 800,
          background_color ='white',
          stopwords = stopwords,
          min_font_size = 10).generate(comment_words)

  # plot the WordCloud image
  print('\n')
  print(color.BOLD +'Common Words in Negative Reviews'+ color.END)
  				
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  plt.title("Common Words in Negative Reviews")	

  plt.show()


# Imput any app lonk from Apple App store 🌷
https://www.apple.com/app-store/

In [16]:
# https://apps.apple.com/us/app/hula-find-events-make-friends/
# https://apps.apple.com/us/app/piccollage-grid-story-maker/id448639966

print(' Apple Store App Link of your choice 🌷 :', '\n', '(click "Enter" to see the result ✨)', '\n')
link = str(input())
review_organizer(link)

 Apple Store App Link of your choice 🌷 : 
 (click "Enter" to see the result ✨) 

https://apps.apple.com/us/app/piccollage-grid-story-maker/id448639966


2022-12-08 14:21:21,822 [INFO] Base - Initialised: AppStore('us', 'piccollage-grid-story-maker', 448639966)
2022-12-08 14:21:21,824 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/piccollage-grid-story-maker/id448639966
2022-12-08 14:21:22,316 [INFO] Base - [id:448639966] Fetched 40 reviews (40 fetched in total)


 Positive Reviews% in this app is 70.00%   
 Negative Reviews% in this app is 30.00%   


 Top 5 positive reviews score:
7.625
7.125
6.875
4.375
4.375
 Top 5 negative reviews score:
-5.375
-3.125
-2.375
-2.25
-2.0

  Top 5 positive reviews score:
review 1 : whenever i want to closely compare a few photos deciding which i like best its much easier to throw them up on this app just to get a better look at them side by side so much easier than flipping back and forth in your photo library 

plus of course you can make cool layouts of pics and texts if you want and easily post on social medial sites this app allows you to use either your own free form style or opt for their already set up grid options very easy when ive got a lot of pics i wanna share with friends from a vacation etc and im sure they dont really want to bear with me through a whole bunch of them  instead ill make a few interesting photo collages with maybe five to eight pics on each layout that way they can get a good feel

ModuleNotFoundError: No module named 'wordcloud'